In [1]:
import os
import time

import pandas as pd 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, countDistinct

import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
!pip show pyspark

Name: pyspark
Version: 3.5.7
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: C:\Users\lento\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: py4j
Required-by: 


# **Spark**

- [스파크에 대해서](https://www.blog.data101.io/304)
- [스파크 설치하기](https://dslyh01.tistory.com/4)

In [3]:
# Spark 세션 만들기
start = time.time()
spark = SparkSession.builder \
    .appName("CheckSpark") \
    .master("local[*]") \
    .config("spark.executor.memory", "4g") \
    .config("spark.ui.showConsoleProgress", "false") \
    .getOrCreate()
end = time.time()

print("Spark session created.", end - start, "seconds elapsed.")

Spark session created. 9.809705018997192 seconds elapsed.


In [4]:
# Spark 세션 출력
spark

In [5]:
# Spark 세션이 존재하는지 확인
spark_session = SparkSession.getActiveSession()

if spark_session is not None:
    print("Spark session is active.")

else:
    print("No active Spark session.")

Spark session is active.


# **데이터 불러오기**

In [6]:
# aggragate 폴더 내 모든 CSV 파일 불러오기
DATA_DIR = "./data/aggregate"

# 해당 디렉토리에서 csv 파일만 가져오기
csv_files = [
    os.path.join(DATA_DIR, f)
    for f in os.listdir(DATA_DIR)
    if f.endswith(".csv")
]

In [7]:
# 여러 CSV 파일을 한 번에 읽기
df = spark.read.csv(csv_files, header=True, inferSchema=True)

# **데이터 오버뷰**

In [8]:
# 데이터프레임의 스키마 출력
df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- game_size: integer (nullable = true)
 |-- match_id: string (nullable = true)
 |-- match_mode: string (nullable = true)
 |-- party_size: integer (nullable = true)
 |-- player_assists: integer (nullable = true)
 |-- player_dbno: integer (nullable = true)
 |-- player_dist_ride: double (nullable = true)
 |-- player_dist_walk: double (nullable = true)
 |-- player_dmg: integer (nullable = true)
 |-- player_kills: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_survive_time: double (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_placement: integer (nullable = true)



In [9]:
# 데이터프레임의 첫 5개 행 출력
df.show(5)

+-------------------+---------+--------------------+----------+----------+--------------+-----------+------------------+------------------+----------+------------+-----------+-------------------+-------+--------------+
|               date|game_size|            match_id|match_mode|party_size|player_assists|player_dbno|  player_dist_ride|  player_dist_walk|player_dmg|player_kills|player_name|player_survive_time|team_id|team_placement|
+-------------------+---------+--------------------+----------+----------+--------------+-----------+------------------+------------------+----------+------------+-----------+-------------------+-------+--------------+
|2017-11-27 05:59:40|       37|2U4GBNA0YmnNZYkzj...|       tpp|         2|             0|          1|          2870.724|        1784.84778|       117|           1|   SnuffIes|            1106.32|      4|            18|
|2017-11-27 05:59:40|       37|2U4GBNA0YmnNZYkzj...|       tpp|         2|             0|          1|2938.4072300000003|1756

In [10]:
# 데이터프레임의 요약 통계 출력
# pandas : df.describe()
df.describe().show()

+-------+------------------+--------------------+----------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+----------------+-------------------+------------------+------------------+
|summary|         game_size|            match_id|match_mode|        party_size|     player_assists|       player_dbno|  player_dist_ride|  player_dist_walk|        player_dmg|      player_kills|     player_name|player_survive_time|           team_id|    team_placement|
+-------+------------------+--------------------+----------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+----------------+-------------------+------------------+------------------+
|  count|          67369231|            67369231|  67369231|          67369231|           67369231|          67369231|          67369231|          67369231|          67369231|          67369

In [11]:
# 레코드 수, 컬럼 수 확인
print(f"Number of records: {df.count()}")
print(f"Number of columns: {len(df.columns)}")

# 컬럼명 확인
print("Columns:", df.columns)

Number of records: 67369231
Number of columns: 15
Columns: ['date', 'game_size', 'match_id', 'match_mode', 'party_size', 'player_assists', 'player_dbno', 'player_dist_ride', 'player_dist_walk', 'player_dmg', 'player_kills', 'player_name', 'player_survive_time', 'team_id', 'team_placement']


In [12]:
# 각 컬럼별 null 개수
df.select([
    sum(col(c).isNull().cast("int")).alias(c)
    for c in df.columns
]).show()

+----+---------+--------+----------+----------+--------------+-----------+----------------+----------------+----------+------------+-----------+-------------------+-------+--------------+
|date|game_size|match_id|match_mode|party_size|player_assists|player_dbno|player_dist_ride|player_dist_walk|player_dmg|player_kills|player_name|player_survive_time|team_id|team_placement|
+----+---------+--------+----------+----------+--------------+-----------+----------------+----------------+----------+------------+-----------+-------------------+-------+--------------+
|   0|        0|       0|         0|         0|             0|          0|               0|               0|         0|           0|      97653|                  0|      0|             0|
+----+---------+--------+----------+----------+--------------+-----------+----------------+----------------+----------+------------+-----------+-------------------+-------+--------------+



# **데이터 오버뷰 한 눈에 보기**

In [14]:
def spark_df_report(df, num_rows: int = 5):
    print("=== DataFrame Overview ===")
    print(f"Row count: {df.count()}")
    print(f"Column count: {len(df.columns)}")
    print(f"Columns: {df.columns}")
    print()
    
    print("=== Schema ===")
    df.printSchema()
    print()
    
    print("=== Sample Data ===")
    df.show(n=num_rows, truncate=False)   # ✅ num_rows를 명시적으로 전달
    print()
    
    print("=== Descriptive Statistics ===")
    df.describe().show()
    print()
    
    print("=== Missing Values per Column ===")
    df.select([
        sum(col(c).isNull().cast("int")).alias(c)
        for c in df.columns
    ]).show()
    print()

In [15]:
spark_df_report(df)

=== DataFrame Overview ===
Row count: 67369231
Column count: 15
Columns: ['date', 'game_size', 'match_id', 'match_mode', 'party_size', 'player_assists', 'player_dbno', 'player_dist_ride', 'player_dist_walk', 'player_dmg', 'player_kills', 'player_name', 'player_survive_time', 'team_id', 'team_placement']

=== Schema ===
root
 |-- date: timestamp (nullable = true)
 |-- game_size: integer (nullable = true)
 |-- match_id: string (nullable = true)
 |-- match_mode: string (nullable = true)
 |-- party_size: integer (nullable = true)
 |-- player_assists: integer (nullable = true)
 |-- player_dbno: integer (nullable = true)
 |-- player_dist_ride: double (nullable = true)
 |-- player_dist_walk: double (nullable = true)
 |-- player_dmg: integer (nullable = true)
 |-- player_kills: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_survive_time: double (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_placement: integer (nullable = true)


=== Sample

# **데이터 컬럼명 의미**

# **데이터 EDA**

In [16]:
df.groupBy("match_mode").count().orderBy("count", ascending=False).show()

+----------+--------+
|match_mode|   count|
+----------+--------+
|       tpp|67369231|
+----------+--------+



해당 데이터는 TPP (3인칭 시점) 게임으로 이루어진 매치들로만 구성되어 있음.

In [17]:
df.groupBy("party_size").count().orderBy("count", ascending=False).show()

+----------+--------+
|party_size|   count|
+----------+--------+
|         4|31721314|
|         2|21608268|
|         1|14039649|
+----------+--------+



전체 중 약 3천만건 이상에서 Squad(4인 파티) 매칭을 참여함.

솔로 플레이는 약 1400만건으로, 사람들이 게임을 홀로 즐기는 사람도 어느 정도 있다는 것으로 확인.